# Log Encodings

Declare4Py provides several among the main encoding techniques for vectorizing a traces log. These are useful for applying Machine Learning techniques. The encoding classes provided by Declare4Py (see the `Declare4Py.Encodings` package) take as an input a log in a Pandas dataframe format and return a Pandas dataframe whose rows represent a single trace and the columns the extracted features. The Declare4Py encodings are implemented as scikit-learn transformers so it is straightfoward to use them in a Machine Learning pipeline.

The tutorial will cover the following points:

1. Encodings families:
    1. The boolean encoding;
    2. The frequency-based encoding;
    3. Aggregated encodings;
    4. Indexed encodings:
        1. The simple-index encoding;
        2. The complex-Index encoding;
    5. Static Encodings:
        1. The first-state encoding;
        2. The second-to-last-state encoding;
        3. The last-state encoding;
    6. The Ngram encoding;
    7. The Declare encoding;
2. Encoding combinations:
    1. The index-latest-payload encoding;
3. A Machine Learning pipeline.

Before starting how to use the encodings the necessary packages need to be imported.

[1]
[2]
[3]
[4]

In [1]:
import os
import pm4py
import pandas as pd


from Declare4Py.Encodings.Aggregate import Aggregate
from Declare4Py.Encodings.IndexBased import IndexBased
from Declare4Py.Encodings.Static import Static
from Declare4Py.Encodings.PreviousState import PreviousState
from Declare4Py.Encodings.LastState import LastState
from Declare4Py.Encodings.Ngram import Ngram
from Declare4Py.Encodings.Declare import Declare

The input format for the `Encodings` classes are logs as Pandas dataframe. Therefore, we import the event log and convert it in a Pandas dataframe.

In [2]:
from Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("../../../", "tests", "test_logs", "Sepsis Cases.xes.gz")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)
case_id_key = event_log.get_case_name()
event_log.to_dataframe()
df = event_log.log
df.head()

## Encodings families

A Declare4Py encoding is implemented as a scikit-learn transformer class, you just need to instantiate the corresponding `encoder` object and call the function `fit_transform(df)` on the input dataframe. The name of the features can be retrieved with the `get_feature_names()` function.

### The Boolean Encoding

In the __boolean encoding__ sequences of events are represented as feature vectors, in such a way that each feature corresponds to an event class (an activity) from the log. This is achieved with the `Declare4Py.Encodings.Aggregate.Aggregate` class by setting the categorical attributes and the `boolean` parameter to `True`.

In [13]:
encoder = Aggregate(case_id_col=case_id_key, cat_cols=['concept:name', 'org:group'], boolean=True)
enc_df = encoder.fit_transform(df)

print(f"Log features:\n {encoder.get_feature_names()}")
enc_df.head()

### The Frequency-Based Encoding

The __frequency-based encoding__, instead of boolean values, represents the control flow in a case with the frequency of each event class in the case. This is achieved with the `Declare4Py.Encodings.Aggregate.Aggregate` class by setting the categorical attributes and the `boolean` parameter to `False`.

In [14]:
encoder = Aggregate(case_id_col=case_id_key, cat_cols=['concept:name', 'org:group'], boolean=False)
enc_df = encoder.fit_transform(df)
enc_df.head()

### The Aggregated Encoding

The __aggregated encoding__ considers all events since the beginning of the case, but ignore the order of the events. In this case, several aggregation functions can be applied to the values that an event attribute has taken throughout the case. This is achieved with the `Declare4Py.Encodings.Aggregate.Aggregate` class by setting the categorical attributes, the numeric attributes, the `boolean` parameter to `False` and a list of functions to aggregate the numeric attributes, e.g., 'mean', 'max', 'min', 'sum', 'std'.

In [15]:
encoder = Aggregate(case_id_col=case_id_key, cat_cols=['concept:name', 'org:group'], num_cols=['CRP'], boolean=False, aggregation_functions=['min', 'mean', 'max'])
enc_df = encoder.fit_transform(df)
enc_df.head()

### Indexed Encodings

#### The Simple-Index Encoding

Another way of encoding a sequence is by taking into account also information about the order in which events occur in the sequence, as in the __simple-index encoding__. Here, each feature corresponds to a position in the sequence and the possible values for each feature are the presence of that event classes. This is achieved with the `Declare4Py.Encodings.IndexBased.IndexBased` class by setting the categorical attributes, the `create_dummies` parameter to `True` and the `max_events` to an integer value lower or equal than the maximum number of events in a trace in the log. If None, the parameter will set to the maximum number of events in a trace in the log. Such parameter sets the first events in the log to be use for indexing.

In [16]:
# with max_events the maximum number of events in a trace in the log.
encoder = IndexBased(case_id_col=case_id_key, cat_cols=['concept:name'], create_dummies=True)
enc_df = encoder.fit_transform(df)
enc_df.head()

In [17]:
# with max_events equal to 2.
encoder = IndexBased(case_id_col=case_id_key, cat_cols=['concept:name'], max_events=2, create_dummies=True)
enc_df = encoder.fit_transform(df)
enc_df.head()

#### The Complex-Index Encoding

The __complex-based encoding__ takes into account also payload columns in the `cat_cols` or `num_cols`  parameters.

In [18]:
encoder = IndexBased(case_id_col=case_id_key, cat_cols = ['concept:name', 'org:group'], num_cols=['CRP'], create_dummies=True)
enc_df = encoder.fit_transform(df)
enc_df.head()

### Static Encodings

In a static encoding, only an available snapshot of the data is used. Therefore, the size of the feature vector is proportional to the number of event attributes and is fixed throughout the execution of a case.

Using the last state abstraction, only one value (e.g., the last snapshot) of each data attribute is available. Here, the numeric attributes are added to the feature vector "as is" while one hot encoding is applied to each categorical attribute.

#### The First-State Encoding
In the __first-state encoding__ only the information (control flow and payload) of the first event is retained. This is achieved with the `Declare4Py.Encodings.Static.Static` class by setting the categorical and numeric attributes.

In [19]:
encoder = Static(case_id_col=case_id_key, cat_cols = ['concept:name', 'org:group'], num_cols=['CRP'])
enc_df = encoder.fit_transform(df)
enc_df.head()

#### The Second-to-Last-State Encoding

In the __second-to-last-state encoding__ only the information (control flow and payload) of the second-to-last event is retained. This is achieved with the `Declare4Py.Encodings.PreviousState.PreviousState` class by setting the categorical and numeric attributes.

In [20]:
encoder = PreviousState(case_id_col=case_id_key, cat_cols = ['concept:name', 'org:group'], num_cols=['CRP'])
enc_df = encoder.fit_transform(df)
enc_df.head()

#### The Last-State Encoding

In the __last-state encoding__ only the information (control flow and payload) of the last event is retained. This is achieved with the `Declare4Py.Encodings.LastState.LastState` class by setting the categorical and numeric attributes.

In [21]:
encoder = LastState(case_id_col=case_id_key, cat_cols = ['concept:name', 'org:group'], num_cols=['CRP'])
enc_df = encoder.fit_transform(df)
enc_df.head()

### The Ngram encoding

In [22]:
encoder = Ngram(case_id_col=case_id_key, n=2 , v=0.7, act_col='concept:name')
enc_df = encoder.fit_transform(df)
enc_df.head()

### The Declare encoding

In [23]:
encoder = DeclareTransformer(case_id_col=case_id_key, n=3 , v= 0.7, act_col='concept:name')
enc_df = encoder.fit_transform(df)
enc_df.head()

## Encoding combinations

### The Index-Latest-Payload Encoding

The index latest-payload encoding adds the lat- est encoding to the simple-index encoding.

combination of a index-based encoding with a static one (the last state)

In [3]:
last_state_encoder = LastStateTransformer(case_id_col=case_id_key, cat_cols=['org:group'], num_cols=[])
latest_df = last_state_encoder.fit_transform(df)

simple_index_encoder = IndexBasedTransformer(case_id_col=case_id_key, cat_cols=['concept:name'], num_cols=[], create_dummies=True)
simple_df = simple_index_encoder.fit_transform(df)

index_latest_payload_df = pd.concat([latest_df, simple_df], axis=1)
index_latest_payload_df.head()

## A Machine Learning pipeline


Esempio di pipeline per variant discovery basata su CF

### TODO: mettere in un df trace id e label
### TODO fare clustering su varianti
### TODO mostra 2 tracce con stesse label hanno variante simile, e due classi con lbl diversa hanno diverse varianti

In [51]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

variants_discovery = Pipeline([('vect', Aggregate(case_id_col=case_id_key, cat_cols=['concept:name'], num_cols=[], boolean=True)),
                              ('kmeans', KMeans(n_clusters=3, random_state=0))])
variants_discovery.fit_transform(df)

for label in discover_variants['kmeans'].labels_:
    print(label)